In [1]:
%pip install artifacts/gpt_wrapper-0.0.8-py3-none-any.whl
%pip install tiktoken
%pip install datasets
%pip install wikipedia-api

Processing ./artifacts/gpt_wrapper-0.0.8-py3-none-any.whl
gpt-wrapper is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import gpt_wrapper
from gpt_wrapper.chat import Chat
import wikipediaapi
import random
import time
from IPython.display import clear_output

In [3]:
gpt_wrapper.api_key = "KEY_REMOVED_FOR_PRIVACY"

In [4]:
# Load JSON file
def load_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data

In [5]:
solutions = load_json('solutions_v1.json')

In [6]:
def extract_keywords_from_response(response):
    keywords = []
    for line in response.split('\n'):
        if line.strip() != "":
            try:
                number, keyword = line.split(". ", 1)
                # verify that the number is an integer
                int(number)
                keywords.append(keyword.strip())
            except ValueError:
                # the line was not in the expected format
                return None
    return keywords

In [7]:
prompts_FR = [
    "Veuillez définir le concept de",
    "Pouvez-vous donner une définition de",
    "Quelle est la signification de",
    "J'aimerais connaître la définition de",
    "Pouvez-vous expliquer ce que l'on entend par",
    "Définissez le terme",
    "Que signifie le terme",
    "Pouvez-vous me donner une brève explication de",
    "Veuillez décrire le concept de",
    "Quelle est la définition de"
]

prompts_EN = [
    "Please define the concept of",
    "Can you provide a definition for",
    "What is the meaning of",
    "I would like to know the definition of",
    "Could you explain what is meant by",
    "Define the term",
    "What does the term",
    "Can you give me a brief explanation of",
    "Please describe the concept of",
    "What is the definition of"
]

In [8]:
import json

datapoint = {
    "Question": "Pouvez-vous me donner une br\u00e8ve explication de Rayonnement \u00e9lectromagn\u00e9tique",
    "Answer": "Le rayonnement \u00e9lectromagn\u00e9tique est  une forme de transfert d'\u00e9nergie lin\u00e9aire. La lumi\u00e8re visible est un rayonnement \u00e9lectromagn\u00e9tique, mais ne constitue qu'une petite tranche du large spectre \u00e9lectromagn\u00e9tique. La propagation de ce rayonnement, d'une ou plusieurs particules, donne lieu \u00e0 de nombreux ph\u00e9nom\u00e8nes comme l'att\u00e9nuation, l'absorption, la diffraction et la r\u00e9fraction, le d\u00e9calage vers le rouge, les interf\u00e9rences, les \u00e9chos, les parasites \u00e9lectromagn\u00e9tiques et les effets biologiques.\nLe rayonnement \u00e9lectromagn\u00e9tique peut \u00eatre d\u00e9crit de mani\u00e8re corpusculaire comme la propagation de photons (boson vecteur de l'interaction \u00e9lectromagn\u00e9tique), ou de mani\u00e8re ondulatoire comme une onde \u00e9lectromagn\u00e9tique. Il se manifeste sous la forme d'un champ \u00e9lectrique coupl\u00e9 \u00e0 un champ magn\u00e9tique.",
    "ID": 2
}

json_string = json.dumps(datapoint, ensure_ascii=False)
decoded_datapoint = json.loads(json_string)
print(decoded_datapoint)

{'Question': 'Pouvez-vous me donner une brève explication de Rayonnement électromagnétique', 'Answer': "Le rayonnement électromagnétique est  une forme de transfert d'énergie linéaire. La lumière visible est un rayonnement électromagnétique, mais ne constitue qu'une petite tranche du large spectre électromagnétique. La propagation de ce rayonnement, d'une ou plusieurs particules, donne lieu à de nombreux phénomènes comme l'atténuation, l'absorption, la diffraction et la réfraction, le décalage vers le rouge, les interférences, les échos, les parasites électromagnétiques et les effets biologiques.\nLe rayonnement électromagnétique peut être décrit de manière corpusculaire comme la propagation de photons (boson vecteur de l'interaction électromagnétique), ou de manière ondulatoire comme une onde électromagnétique. Il se manifeste sous la forme d'un champ électrique couplé à un champ magnétique.", 'ID': 2}


In [9]:
debug = False

In [10]:
# Create an empty list to store the datapoints
QandA_EN = []
QandA_FR = []
start_index = 0

try:
    # Load the previous list of datapoints (useful to resume the process if it was interrupted)
    with open('QandA_definitions_EN.json', 'r') as f:
        QandA_EN = json.load(f)
    with open('QandA_definitions_FR.json', 'r') as f:
        QandA_FR = json.load(f)

    # Find the highest ID among the existing datapoints
    start_index_EN = max([int(item['ID']) for item in QandA_EN]) + 1 if QandA_EN else -1
    start_index_FR = max([int(item['ID']) for item in QandA_FR]) + 1 if QandA_FR else -1

    # Check if the starting indices match
    if start_index_EN != start_index_FR:
        print("Starting indices don't match between English and French datasets.")
        if start_index_EN > start_index_FR:
            start_index = start_index_EN
        else:
            start_index = start_index_FR
        print("Starting index:", start_index)
    elif start_index_EN == -1:
        print("Error: No datapoints found in the existing datasets.")
    else:
        start_index = start_index_EN
        print("Starting index:", start_index)

except Exception as e:
    print(f"Failed to load previous data: {e}")
    
data_len = len(solutions)
start_time = time.time()
elapsed_times = []
token_costs = []

for i, datapoint in enumerate(solutions[start_index:], start_index):

    iteration_start_time = time.time()
    cost = 0

    question = datapoint['question']
    if question is not None:
        max_retries = 3
        for j in range(max_retries):
            chat = Chat.create("EN_Key_topics_" + str(i) + "_" + str(j))
            query = "Please list the key topics related to the following question, each one numbered and on a new line. Each topic should be SHORT (a couple of words per topic MAXIMUM) and ideally correspond to a Wikipedia page title (DO NOT INCLUDE LINKS, DO NOT INCLUDE ABBREVIATIONS). Regardless of the language of the question, return keywords in ENGLISH:\n\n" + question
            used_before = Chat.budget()['usage']
            A = chat.ask(content=query)
            used_after = Chat.budget()['usage']
            cost = used_after - used_before
            
            keywords = extract_keywords_from_response(A.content)
            if keywords is not None:
                count_keywords_EN = len(keywords)
                if debug:
                    print("Keywords:", keywords)
                    print("Number of keywords obtained: ", len(keywords))
                break
            else:
                if debug:
                    print(f"Response was not in the expected format, retrying ({i+1}/{max_retries})")

        # Create a Wikipedia API client
        wiki_wiki = wikipediaapi.Wikipedia('en')

        # Retrieve and print the main definition for each keyword
        datapoint_count_EN = 0
        if keywords is not None:
            for keyword in keywords:
                page = wiki_wiki.page(keyword)
                if page.exists():
                    if "may refer to" in page.text:
                        if debug:
                            print(f"Skipping disambiguation page for '{keyword}'")
                    else:
                        if debug:
                            print(f"Main definition for '{keyword}':")
                            print(page.summary)

                        prompt = random.choice(prompts_EN)
                        datapoint_EN = {
                            "Question": f"{prompt} {keyword}",
                            "Answer": page.summary,
                            "ID": i,
                            "Keyword": keyword,
                            "Language": "EN"
                        }
                        if debug:
                            print(datapoint_EN)
                        datapoint_count_EN += 1

                        QandA_EN.append(datapoint_EN)
                else:
                    if debug:
                        print(f"No webpage found for '{keyword}'")

        #############################################################################
        
        max_retries = 3
        for j in range(max_retries):
            chat = Chat.create("FR_Key_topics_" + str(i) + "_" + str(j))
            query = "Please list the key topics related to the following question, each one numbered and on a new line. Each topic should be SHORT (a couple of words per topic MAXIMUM) and ideally correspond to a Wikipedia page title (DO NOT INCLUDE LINKS, DO NOT INCLUDE ABBREVIATIONS). Regardless of the language of the question, return keywords in FRENCH:\n\n" + question
            used_before = Chat.budget()['usage']
            A = chat.ask(content=query)
            used_after = Chat.budget()['usage']
            cost = cost + (used_after - used_before)
            
            keywords = extract_keywords_from_response(A.content)
            if keywords is not None:
                count_keywords_FR = len(keywords)
                if debug:
                    print("Keywords:", keywords)
                    print("Number of keywords obtained: ", len(keywords))
                break
            else:
                if debug:
                    print(f"Response was not in the expected format, retrying ({i+1}/{max_retries})")

        # Create a Wikipedia API client
        wiki_wiki = wikipediaapi.Wikipedia('fr')

        # Retrieve and print the main definition for each keyword
        datapoint_count_FR = 0
        if keywords is not None:
            for keyword in keywords:
                page = wiki_wiki.page(keyword)
                if page.exists():
                    if "plusieurs concepts" in page.text or "dans les articles suivants" in page.text or "Suivant le contexte, le terme" in page.text:
                        if debug:
                            print(f"Skipping disambiguation page for '{keyword}'")
                    else:
                        if debug:
                            print(f"Main definition for '{keyword}':")
                            print(page.summary)

                        prompt = random.choice(prompts_FR)
                        datapoint_FR = {
                            "Question": f"{prompt} {keyword}",
                            "Answer": page.summary,
                            "ID": i,
                            "Keyword": keyword,
                            "Language": "FR"
                        }
                        if debug:
                            print(datapoint_FR)
                        datapoint_count_FR += 1

                        QandA_FR.append(datapoint_FR)
                else:
                    if debug:
                        print(f"No webpage found for '{keyword}'")

        # Save the list of datapoints as a JSON file
        with open('QandA_definitions_EN.json', 'w') as f:
            json.dump(QandA_EN, f)
        with open('QandA_definitions_FR.json', 'w') as f:
            json.dump(QandA_FR, f)

        iteration_end_time = time.time()

        elapsed_time = iteration_end_time - iteration_start_time
        elapsed_times.append(elapsed_time)

        token_cost = used_after - used_before
        token_costs.append(token_cost)

        average_time_per_datapoint = sum(elapsed_times) / len(elapsed_times)
        remaining_datapoints = data_len - i
        estimated_time_remaining = remaining_datapoints * average_time_per_datapoint

        clear_output()
        print("########################################################")
        print("Processed datapoint", i, "of", (data_len - 1), "(", round(i/(data_len-1)*100, 2), "%)")
        print("########################################################")

        print("EN datapoints: ", datapoint_count_EN, " out of ", count_keywords_EN, " keywords | FR datapoints: ", datapoint_count_FR, " out of ", count_keywords_FR, " keywords")
        print("ETA (minutes): ", round(estimated_time_remaining/60, 2), "| Average tokens used per iteration: ", sum(token_costs) / len(token_costs))
        tokens_remaining = Chat.budget()['limit'] - used_after
        estimated_iterations_remaining = tokens_remaining / ((sum(token_costs) / len(token_costs)))
        print("Estimated iterations remaining: ", estimated_iterations_remaining)

########################################################
Processed datapoint 4449 of 4449 ( 100.0 %)
########################################################
EN datapoints:  4  out of  5  keywords | FR datapoints:  4  out of  5  keywords
ETA (minutes):  0.13 | Average tokens used per iteration:  212.2934451219512
Estimated iterations remaining:  1037.6439078157032
